## Preprocessing

**Interesting links**

https://medium.com/@cd_24/bertopic-fine-tune-parameters-76c3377016fb

**Import Libraries**

In [115]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from umap import UMAP

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer


import pandas as pd
import openpyxl

**Inputs**

In [116]:
# choose from 'multi or 'seznam'
model_type = 'multi'

# choose clustering method: 'k-meand' 'db-scan'
cluster_type = 'k-means'

file_name = 'data_sport_history_chemistry_film_biology.csv'
stop_words = 'input/stop_words_czech.txt'


output_vect = f'output/results-vectorized-{model_type}-{cluster_type}.xlsx'
output_reduced = f'output/results-reduced-{model_type}-{cluster_type}.xlsx'

**Load data**

In [117]:
data = pd.read_csv('input/' + file_name, encoding='utf-8', delimiter=';')
data.head(5)

,sentences
0,Fotbal je populárním sportem ve většině zemí.
1,Tenisový turnaj Wimbledon je jedním z nejstarš...
2,"Běhání je skválý způsob, jak si udržet kondici."
3,"Hokejový zápas může být velmi napínavý, zejmén..."
4,Plavání je vynikající cvičení pro posílení sva...


**Put data in a list**

In [118]:
docs = [item for sublist in data.values.tolist() for item in sublist]
docs[:5]

['Fotbal je populárním sportem ve většině zemí.',
 'Tenisový turnaj Wimbledon je jedním z nejstarších a nejprestižnějších na světě.',
 'Běhání je skválý způsob, jak si udržet kondici.',
 'Hokejový zápas může být velmi napínavý, zejména v prodloužení.',
 'Plavání je vynikající cvičení pro posílení svalů celého těla.']

In [119]:
len(docs)

499

## Set up model

**Define the clustering model**

In [120]:
if cluster_type == 'db-scan':
    cluster_model = HDBSCAN(min_cluster_size=30, 
                            min_samples=10,
                            #gen_min_span_tree=True,
                            #prediction_data=True
                            )


elif cluster_type == 'k-means':
    #does not produce any outliers!!!
    cluster_model = KMeans(n_clusters=8)

**Define Count Vectorizer**

In [121]:
# Read your custom Czech stop words from the CSV file
stopwords_df = pd.read_csv(stop_words, header=None)
czech_stop_words = stopwords_df[0].tolist()


# Convert the set of stop words into a list
czech_stop_words_list = list(czech_stop_words)
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=czech_stop_words_list)

**Define the UMAP**


In [122]:
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)

**Define Sentence Transformer**

In [123]:
 # model-multi
if model_type == 'multi':
    model_name = 'all-MiniLM-L6-v2'
    model = SentenceTransformer(model_name)

# model-Seznam
elif model_type == 'seznam':
    model_name = 'Seznam/retromae-small-cs'
    model = AutoModel.from_pretrained(model_name)

**Define model**

In [124]:
model = BERTopic(#language="czech", # not necessary if sentence transformer is chosen
                 embedding_model = model,
                 min_topic_size = 10,
                 top_n_words = 15,
                 #vectorizer_model=vectorizer_model, 
                 hdbscan_model = cluster_model, 
                 umap_model = umap_model
                 )

## Train model

In [125]:
""" It is not going to work with a few hundereds documents, better to have thousands"""

topics, probs = model.fit_transform(docs)

## Coherence score

In [126]:
#https://github.com/MaartenGr/BERTopic/issues/90

import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd


'''
if error boolean - run the whole notebook again!!!
'''

def coherence_score(model, docs: pd.DataFrame, topics):
    # Merge all sentences for certain topic
    documents = pd.DataFrame({"Document": docs,
                            #"ID": range(len(docs)),
                            "Topic": topics})

    documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
    cleaned_docs = model._preprocess_text(documents_per_topic.Document.values)

    # Extract vectorizer and analyzer from BERTopic
    vectorizer = model.vectorizer_model
    analyzer = vectorizer.build_analyzer()

    # Use '.get_feature_names_out()' if you get an error with '.get_feature_names()'
    words = vectorizer.get_feature_names_out()

    # Extract features for Topic Coherence evaluation
    tokens = [analyzer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]


    # Extract words in each topic if they are non-empty and exist in the dictionary - SIMPLE 
    topic_words = [[words for words, _ in model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

    # Evaluate Coherence
    coherence_model = CoherenceModel(topics=topic_words, 
                                    texts=tokens, 
                                    corpus=corpus,
                                    dictionary=dictionary, 
                                    coherence='c_v')
    coherence = coherence_model.get_coherence()
    return round(coherence,3)

In [127]:
coherence_score(model=model, docs=docs, topics=topics)

0.775

## Visualize results

In [128]:
model.get_topic_freq()

,Topic,Count
2,0,118
0,1,102
6,2,89
3,3,82
1,4,56
4,5,26
5,6,19
7,7,7


In [129]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,118,0_roce_byl_po_století,"[roce, byl, po, století, první, na, ve, české,...",[Třicetiletá válka v 17. století měla tragický...
1,1,102,1_je_sport_sportem_který,"[je, sport, sportem, který, na, oblíbeným, ene...",[Cyklistika je oblíbeným sportem pro rekreační...
2,2,89,2_jsou_jejich_organismů_organismy,"[jsou, jejich, organismů, organismy, zkoumá, s...",[Ekologie zkoumá vztahy mezi organismy a jejic...
3,3,82,3_film_český_získal_za,"[film, český, získal, za, je, filmová, filmy, ...",[Kolya získal Oscara za nejlepší cizojazyčný f...
4,4,56,4_chemická_rychlosti_atomy_reakce,"[chemická, rychlosti, atomy, reakce, molekuly,...","[Chemická rovnováha nastává, když rychlosti př..."
5,5,26,5_buněk_je_proces_prostředí,"[buněk, je, proces, prostředí, bílé, univerzit...",[Karlova univerzita v Praze byla založena v ro...
6,6,19,6_dna_genetické_informace_genetický,"[dna, genetické, informace, genetický, enzymy,...","[Kyseliny nukleové, jako je RNA a DNA, jsou zá..."
7,7,7,7_reakce_exotermické_endotermické_nebo,"[reakce, exotermické, endotermické, nebo, tepl...",[Chemické reakce mohou být exotermické nebo en...


In [130]:
model.visualize_barchart(top_n_topics=10)

In [131]:
model.visualize_topics()

In [132]:
model.visualize_heatmap(top_n_topics=20, width=800, height=800)

## Representation - Count Vectorizer

**Update topics**

In [133]:
model.update_topics(docs, vectorizer_model=vectorizer_model)

**Get counts of the topics**

In [134]:
results_unique_topics = model.get_topic_info()
results_unique_topics[:20]

,Topic,Count,Name,Representation,Representative_Docs
0,0,118,0_roce_století_české_měla,"[roce, století, české, měla, světové, revoluce...",[Třicetiletá válka v 17. století měla tragický...
1,1,102,1_sport_sportem_oblíbeným_energii,"[sport, sportem, oblíbeným, energii, fotosynté...",[Cyklistika je oblíbeným sportem pro rekreační...
2,2,89,2_organismů_organismy_zkoumá_živých,"[organismů, organismy, zkoumá, živých, ekosyst...",[Ekologie zkoumá vztahy mezi organismy a jejic...
3,3,82,3_film_český_získal_filmová,"[film, český, získal, filmová, filmy, český fi...",[Kolya získal Oscara za nejlepší cizojazyčný f...
4,4,56,4_chemická_reakce_atomy_rychlosti,"[chemická, reakce, atomy, rychlosti, rovnováha...","[Chemická rovnováha nastává, když rychlosti př..."
5,5,26,5_buněk_proces_prostředí_střední evropě,"[buněk, proces, prostředí, střední evropě, bíl...",[Karlova univerzita v Praze byla založena v ro...
6,6,19,6_dna_genetické_genetické informace_informace,"[dna, genetické, genetické informace, informac...","[Kyseliny nukleové, jako je RNA a DNA, jsou zá..."
7,7,7,7_reakce_exotermické_endotermické_reakce exote...,"[reakce, exotermické, endotermické, reakce exo...",[Chemické reakce mohou být exotermické nebo en...


**Coherence score**

In [135]:
coherence_score(model=model, docs=docs, topics=topics)

0.679

**Export voctorized data**

In [136]:
# get results into a df
results_vect = model.get_document_info(docs)

# Export to Excel
results_vect.to_excel(output_vect, index=False, engine='openpyxl')

In [137]:
saadsad

NameError: name 'saadsad' is not defined

## Topic reduction

In [138]:
# manually merge topics
topics_to_merge = [(7, 4),
                   (2, 5, 6)
                   ]


model.merge_topics(docs, topics_to_merge)

In [ ]:
""" # Further reduce topics
new_model = model.reduce_topics(docs, nr_topics=5) """

In [139]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,134,0_organismů_organismy_zkoumá_živých,"[organismů, organismy, zkoumá, živých, dna, pr...",[Karlova univerzita v Praze byla založena v ro...
1,1,118,1_roce_století_české_měla,"[roce, století, české, měla, světové, revoluce...",[Česká republika vstoupila do Evropské unie v ...
2,2,102,2_sport_sportem_oblíbeným_energii,"[sport, sportem, oblíbeným, energii, fotosynté...","[Fotosyntéza je proces, při kterém rostliny vy..."
3,3,82,3_film_český_získal_filmová,"[film, český, získal, filmová, filmy, český fi...",[Kolya získal Oscara za nejlepší cizojazyčný f...
4,4,63,4_reakce_chemická_rychlosti_atomy,"[reakce, chemická, rychlosti, atomy, molekuly,...","[Chemická rovnováha nastává, když rychlosti př..."


In [140]:
new_results=model.get_document_info(docs)
new_results.head(100)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,Fotbal je populárním sportem ve většině zemí.,2,2_sport_sportem_oblíbeným_energii,"[sport, sportem, oblíbeným, energii, fotosynté...","[Fotosyntéza je proces, při kterém rostliny vy...",sport - sportem - oblíbeným - energii - fotosy...,False
1,Tenisový turnaj Wimbledon je jedním z nejstarš...,2,2_sport_sportem_oblíbeným_energii,"[sport, sportem, oblíbeným, energii, fotosynté...","[Fotosyntéza je proces, při kterém rostliny vy...",sport - sportem - oblíbeným - energii - fotosy...,False
2,"Běhání je skválý způsob, jak si udržet kondici.",2,2_sport_sportem_oblíbeným_energii,"[sport, sportem, oblíbeným, energii, fotosynté...","[Fotosyntéza je proces, při kterém rostliny vy...",sport - sportem - oblíbeným - energii - fotosy...,False
3,"Hokejový zápas může být velmi napínavý, zejmén...",4,4_reakce_chemická_rychlosti_atomy,"[reakce, chemická, rychlosti, atomy, molekuly,...","[Chemická rovnováha nastává, když rychlosti př...",reakce - chemická - rychlosti - atomy - moleku...,False
4,Plavání je vynikající cvičení pro posílení sva...,2,2_sport_sportem_oblíbeným_energii,"[sport, sportem, oblíbeným, energii, fotosynté...","[Fotosyntéza je proces, při kterém rostliny vy...",sport - sportem - oblíbeným - energii - fotosy...,False
...,...,...,...,...,...,...,...
95,Chemická analýza může odhalit složení látek a ...,0,0_organismů_organismy_zkoumá_živých,"[organismů, organismy, zkoumá, živých, dna, pr...",[Karlova univerzita v Praze byla založena v ro...,organismů - organismy - zkoumá - živých - dna ...,False
96,"Rovnováha v chemii se vyskytuje, když rychlost...",4,4_reakce_chemická_rychlosti_atomy,"[reakce, chemická, rychlosti, atomy, molekuly,...","[Chemická rovnováha nastává, když rychlosti př...",reakce - chemická - rychlosti - atomy - moleku...,False
97,Exotermické reakce uvolňují energii do okolí v...,4,4_reakce_chemická_rychlosti_atomy,"[reakce, chemická, rychlosti, atomy, molekuly,...","[Chemická rovnováha nastává, když rychlosti př...",reakce - chemická - rychlosti - atomy - moleku...,False
98,Chirální molekuly mají zrcadlově symetrické ve...,4,4_reakce_chemická_rychlosti_atomy,"[reakce, chemická, rychlosti, atomy, molekuly,...","[Chemická rovnováha nastává, když rychlosti př...",reakce - chemická - rychlosti - atomy - moleku...,False


In [141]:
new_results.to_excel(output_reduced, index=False, engine='openpyxl')

## Translate

In [ ]:
""" from deep_translator import GoogleTranslator

def translate_sentences(sentences, source_language='cs', target_language='en'):
    translated_sentences = []
    
    for sentence in sentences:
        translated = GoogleTranslator(source=source_language, target=target_language).translate(sentence)
        translated_sentences.append(translated)
    
    return translated_sentences

# Example usage
sentences_to_translate = [
    'snažím se dovolat uz minut',
    'ahoj jak se mas',
    'mam se dobre a ty?'
]

source_language = 'cs'
target_language = 'en'

translated_sentences = translate_sentences(sentences_to_translate, source_language, target_language)

# Print the original and translated sentences
for original, translated in zip(sentences_to_translate, translated_sentences):
    print(f"Original: {original}")
    print(f"Translated: {translated}")
    print() """

' from deep_translator import GoogleTranslator\n\ndef translate_sentences(sentences, source_language=\'cs\', target_language=\'en\'):\n    translated_sentences = []\n    \n    for sentence in sentences:\n        translated = GoogleTranslator(source=source_language, target=target_language).translate(sentence)\n        translated_sentences.append(translated)\n    \n    return translated_sentences\n\n# Example usage\nsentences_to_translate = [\n    \'snažím se dovolat uz minut\',\n    \'ahoj jak se mas\',\n    \'mam se dobre a ty?\'\n]\n\nsource_language = \'cs\'\ntarget_language = \'en\'\n\ntranslated_sentences = translate_sentences(sentences_to_translate, source_language, target_language)\n\n# Print the original and translated sentences\nfor original, translated in zip(sentences_to_translate, translated_sentences):\n    print(f"Original: {original}")\n    print(f"Translated: {translated}")\n    print() '